In [1]:
import petlink32
import numpy as np
import time

In [2]:
#prints from Siemens walkthrough, a bed position tag
ew1 =  0xc40d180c
hbp = 0xd180c
hbp_se = -190452

In [3]:
my_hbp = petlink32.HORIZONTAL_BED_POSITION.evaluate(hbp)
print("hbp diff = {}".format(hbp-my_hbp))

hbp diff = 0


In [4]:
my_hbp_se = petlink32.sign_extend(my_hbp, 20)
print(my_hbp_se)
diff = hbp_se - my_hbp_se
print("hbp_se diff = {}".format(diff))

-190452
hbp_se diff = 0


In [5]:
print(petlink32.sign_extend(np.array([my_hbp]*3), 20))

[-190452 -190452 -190452]


In [13]:
# packet construction from PETLink document
doc_string = ["PET Bin-Address Event Packet", 
              "0PBB BBBB BBBB BBBB BBBB BBBB BBBB BBBB",
              "P: Prompt (1 - Prompt; 0 - Delay)",
              "B: Bin Address: 0-29"] 
packet_name = doc_string[0]
part_names = ["0: Tag bit "] + doc_string[2:]
print(part_names)
packet_instruction = doc_string[1]
packet = petlink32.packet(packet_instruction, packet_name, part_names)
print(packet)
tag = 1 << 31
event = 23 
print("packet compare: event = {}, tag = {}".format(packet.compare(event), packet.compare(tag)))
print("event packet evaluation ", packet.evaluate(event))

['0: Tag bit ', 'P: Prompt (1 - Prompt; 0 - Delay)', 'B: Bin Address: 0-29']
packet(name = PET Bin-Address Event Packet
PETLink_doc_str = 0PBB BBBB BBBB BBBB BBBB BBBB BBBB BBBB
parts = 
0: Tag bit 
(lowbit, highbit) = (31, 31)
value = 0 (base 2)
mask =  1000 0000 0000 0000 0000 0000 0000 0000
---
P: Prompt (1 - Prompt; 0 - Delay)
(lowbit, highbit) = (30, 30)
mask =  0100 0000 0000 0000 0000 0000 0000 0000
---
B: Bin Address: 0-29
(lowbit, highbit) = (0, 29)
mask =  0011 1111 1111 1111 1111 1111 1111 1111)
packet compare: event = True, tag = False


NameError: name 'value_parts' is not defined

In [7]:
doc_string = "Position Profile Data (Maintenance) \n ...
0uuu uHHH BBBB BBBh XXXX XXXX YYYY YYYY \n ...
u: Undefined (Default: Set to zero.) \n
H: Head number: 0-2 [See 3 H-field bits labelled by \"210 \" above.] \n
B: Block position: 0-6 \n
h: Hit bit (always = 1) \n
X: X position data: 0-7 \n
Y: Y position data: 0-7 \n "

SyntaxError: EOL while scanning string literal (<ipython-input-7-156e6c86131f>, line 1)